In [ ]:
#Table of content

#1 Prediction 1
#1.1 Fast GBM
#1.2 Load libraries
#1.3 Reading in data 
#1.4 Data pre-processing
#1.5 Split train/test
#1.6 Fast GBM 1
#1.7 Fast GBM 2 
#1.8 Fast GBM 3
#1.9 Prediction

#2 Prediction 2 
#2.1 Auto modelling
#2.2 Imputation
#2.3 Hyper-parameter tuning
#2.4 Optimized predictive model GBM 
#2.5 Prediction

#3 Prediction 3
#3.1 GBM with all variables 
#3.2 Fast GBM 4 
#3.3 Prediction

#4 Prediction 4
#4.1 Ensembling
#4.2 Stacking ensemble

#5 Prediction 5
#5.1 EDA in-depth analysis
#5.2 Descriptive statistics
#5.3 The Y/response variable
#5.4 Correlation
#5.5 PCA full dataframe
#5.6 PCA reduced dataframe
#5.7 PCA PC's automatically selected

#6 Prediction 6
#6.1 Lean XG Boost
#6.2 Prediction


In [ ]:
######################################################################################################################
######################################################################################################################
##1) Prediction 1  

#1.1) Fast GBM 

#First we will apply a fast GBM for quick first results and valuable insight in the data.

In [ ]:
######################################################################################################################
######################################################################################################################
##1.2) Load libraries

# data analysis and wrangling
import numpy as np
import random as rnd
import pandas as pd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

from collections import Counter
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_squared_error,roc_auc_score,precision_score

from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve 

from sklearn.metrics import classification_report
from sklearn.model_selection import learning_curve,GridSearchCV
from sklearn.metrics import confusion_matrix

from sklearn.preprocessing import LabelEncoder
import random


In [ ]:
######################################################################################################################
######################################################################################################################
##1.3) Reading in data 

# Reading in data
uploaded = files.upload()
# Storing data in a Pandas dataframe
train = pd.read_csv(io.BytesIO(uploaded['train.csv']))

In [ ]:
######################################################################################################################
######################################################################################################################
##1.4) Data pre-processing

train = pd.DataFrame(train)
print("Original DataFrame:")
print(train)
train = train.sample(frac=1)
print("\nNew DataFrame:")
print(train)

#Subset data
train.iloc[0:20000]

train.dtypes

train['TARGET'].dtypes

myvars = train[["TARGET","var3","var15","var38"]]
train1 = myvars

myvars

train.isnull().sum().sum()
train.isnull().sum()

sns.catplot(x="TARGET", kind="count", data=train1)

## Fast feature importance (Feature importance = probably the most valuable result you can get, 
# So you want to get it as fast as possible)
X = train
Y_train = X["TARGET"]
X_train = X.drop("TARGET", axis=1)

model = LGBMClassifier()
model.fit(X_train, Y_train)

fig, ax = plt.subplots(figsize=(12,18))
lgb.plot_importance(model, max_num_features=50, height=0.8, ax=ax)
ax.grid(False)
plt.title("LightGBM - Feature Importance", fontsize=15)
plt.show()

## Fast feature importance (advanced)

## # Split dataset and dependent
## Y_train = train["TARGET"]
## X_train = train.drop("TARGET", axis=1)
## OR
## Y_train = train["TARGET"]
## X_train = train
## #scaling the features using Standard Scaler
## sc=StandardScaler()
## sc.fit(X_train)
## X=pd.DataFrame(sc.fit_transform(X_train))
## #train_test_split 
## X_train,X_test,y_train,y_test=train_test_split(X,Y_train,test_size=0.3,random_state=0)
## #converting the dataset into proper LGB format 
## d_train=lgb.Dataset(X_train, label=y_train)
## #Specifying the parameter
## params={}
## params['learning_rate']=0.03
## params['boosting_type']='gbdt' #GradientBoostingDecisionTree
## params['objective']='binary' #Binary target feature
## params['metric']='binary_logloss' #metric for binary classification
## params['max_depth']=10
## #train the model 
## model=lgb.train(params,d_train,100) #train the model on 100 epocs
## #prediction on the test set
## # Feature importance
## y_pred=model.predict(X_test)
## fig, ax = plt.subplots(figsize=(12,18))
## lgb.plot_importance(model, max_num_features=50, height=0.8, ax=ax)
## ax.grid(False)
## plt.title("LightGBM - Feature Importance", fontsize=15)
## plt.show()


In [ ]:
######################################################################################################################
######################################################################################################################
##1.5) Split train/test

# split into train test sets
X = train
X_train, X_test= train_test_split(X, train_size=0.70)
print(X_train.shape, X_test.shape)

# split features and dependent
Y_train = X_train["TARGET"]
X_train = X_train.drop("TARGET", axis=1)
Y_test = X_test["TARGET"]
X_test = X_test.drop("TARGET", axis=1)

In [ ]:
######################################################################################################################
######################################################################################################################
##1.6) Fast GBM 1

# GBM classifier 1
model = GradientBoostingClassifier(
        learning_rate=0.1, 
        n_estimators=100,
        max_depth=3, 
        min_samples_split=2, 
        min_samples_leaf=1, 
        subsample=1,
        max_features='sqrt', 
        random_state=10)

model.fit(X_train,Y_train)

pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

accuracy_train = accuracy_score(pred_train,Y_train)
accuracy_test = accuracy_score(pred_test,Y_test)

fpr, tpr, _ = metrics.roc_curve(np.array(Y_train), model.predict_proba(X_train)[:,1])
auc_train = metrics.auc(fpr,tpr)

fpr, tpr, _ = metrics.roc_curve(np.array(Y_test), model.predict_proba(X_test)[:,1])
auc_test = metrics.auc(fpr,tpr)

## Cross validation accuracy
cross_val_rfc = cross_val_score(estimator=GradientBoostingClassifier(), X=X_train, y=Y_train, cv=2, n_jobs=-1)

# R Cross validation accuracy
cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=1, random_state=1)
# evaluate the model on the dataset
n_scores = cross_val_score(model, X_train, Y_train, scoring='accuracy', cv=cv, n_jobs=-1)

#Print model report:
print("\nModel Report")
print('AUC:',(auc_test))
print("Cross Validation Accuracy : ",round(cross_val_rfc.mean() * 100 , 2),"%")
print(' R Cv Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
print("Cross Validation Accuracy: ")
print(classification_report(Y_test, pred_test))

In [ ]:
######################################################################################################################
######################################################################################################################
##1.6) Fast GBM 2

In [ ]:
# Exclude the ID column
X = train.drop("ID", axis=1)

# Split train/test
X_train, X_test= train_test_split(X, train_size=0.70)
print(X_train.shape, X_test.shape)

# Split features/dependent
Y_train = X_train["TARGET"]
X_train = X_train.drop("TARGET", axis=1)
Y_test = X_test["TARGET"]
X_test = X_test.drop("TARGET", axis=1)


In [ ]:
# GBM classifier 2
model = GradientBoostingClassifier(
        learning_rate=0.1, 
        n_estimators=100,
        max_depth=3, 
        min_samples_split=2, 
        min_samples_leaf=1, 
        subsample=1,
        max_features='sqrt', 
        random_state=10)

model.fit(X_train,Y_train)

pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

accuracy_train = accuracy_score(pred_train,Y_train)
accuracy_test = accuracy_score(pred_test,Y_test)

fpr, tpr, _ = metrics.roc_curve(np.array(Y_train), model.predict_proba(X_train)[:,1])
auc_train = metrics.auc(fpr,tpr)

fpr, tpr, _ = metrics.roc_curve(np.array(Y_test), model.predict_proba(X_test)[:,1])
auc_test = metrics.auc(fpr,tpr)

## Cross validation accuracy
cross_val_rfc = cross_val_score(estimator=GradientBoostingClassifier(), X=X_train, y=Y_train, cv=2, n_jobs=-1)

# R Cross validation accuracy
cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=1, random_state=1)
# evaluate the model on the dataset
n_scores = cross_val_score(model, X_train, Y_train, scoring='accuracy', cv=cv, n_jobs=-1)

#Print model report:
print("\nModel Report")
print('AUC:',(auc_test))
print("Cross Validation Accuracy : ",round(cross_val_rfc.mean() * 100 , 2),"%")
print(' R Cv Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
print("Cross Validation Accuracy: ")
print(classification_report(Y_test, pred_test))


In [ ]:
######################################################################################################################
######################################################################################################################
##1.8) Fast GBM 3 

In [ ]:
#Creating clean dataset
myvars = train[["TARGET","var3","var15","var38"]]
train1 = myvars

# Split train/test
X_train, X_test= train_test_split(train1, train_size=0.70)
print(X_train.shape, X_test.shape)

# Split features/dependent
Y_train = X_train["TARGET"]
X_train = X_train.drop("TARGET", axis=1)
Y_test = X_test["TARGET"]
X_test = X_test.drop("TARGET", axis=1)

In [ ]:
# GBM classifier 3
model = GradientBoostingClassifier(
        learning_rate=0.1, 
        n_estimators=100,
        max_depth=3, 
        min_samples_split=2, 
        min_samples_leaf=1, 
        subsample=1,
        max_features='sqrt', 
        random_state=10)

model.fit(X_train,Y_train)

pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

accuracy_train = accuracy_score(pred_train,Y_train)
accuracy_test = accuracy_score(pred_test,Y_test)

fpr, tpr, _ = metrics.roc_curve(np.array(Y_train), model.predict_proba(X_train)[:,1])
auc_train = metrics.auc(fpr,tpr)

fpr, tpr, _ = metrics.roc_curve(np.array(Y_test), model.predict_proba(X_test)[:,1])
auc_test = metrics.auc(fpr,tpr)

## Cross validation accuracy
cross_val_rfc = cross_val_score(estimator=GradientBoostingClassifier(), X=X_train, y=Y_train, cv=2, n_jobs=-1)

# R Cross validation accuracy
cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=1, random_state=1)
# evaluate the model on the dataset
n_scores = cross_val_score(model, X_train, Y_train, scoring='accuracy', cv=cv, n_jobs=-1)

#Print model report:
print("\nModel Report")
print('AUC:',(auc_test))
print("Cross Validation Accuracy : ",round(cross_val_rfc.mean() * 100 , 2),"%")
print(' R Cv Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
print("Cross Validation Accuracy: ")
print(classification_report(Y_test, pred_test))


In [ ]:
## 1.9 Prediction

## ROC/AUC 

print("Roc : ",(fpr, tpr, _))
print("Auc : ",(auc_test))

## Confusion matrix
print("Confusion matrix : ", confusion_matrix(Y_test, pred_test))

## Crosstab
pd.crosstab(Y_train,pd.Series(pred_train),rownames=['ACTUAL'],colnames=['PRED'])

## cross validation accuracy
print("Cross Validation Accuracy : ",round(cross_val_rfc.mean() * 100 , 2),"%")

## R cross validation accuracy
print(' R Cv Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

# ROC AUC plot
#baseline model
baseline_roc_auc = roc_auc_score(Y_test, model.predict(X_test))
fprB, tprB, thresholdsB = roc_curve(Y_test, model.predict_proba(X_test)[:,1])
## #model 1
## model1_roc_auc = roc_auc_score(y_test, model1.predict(X_test))
## fpr1, tpr1, thresholds1 = roc_curve(y_test, model1.predict_proba(X_test)[:,1])
## #new tuned model 
## new_roc_auc = roc_auc_score(y_test, new.predict(X_test))
## fprnew, tprnew, thresholds_new = roc_curve(y_test, new.predict_proba(X_test)[:,1])

plt.figure()
plt.plot(fprB, tprB, label='GBM Baseline (area = %0.2f)' % baseline_roc_auc)
## plt.plot(fpr1, tpr1, label='GBM Model 1 (area = %0.2f)' % model1_roc_auc)
## plt.plot(fprnew, tprnew, label='GBM Final Model (area = %0.2f)' % new_roc_auc)

plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

In [ ]:
######################################################################################################################
######################################################################################################################
##2) Prediction 2 
                            
#2.1) Auto-modelling 
                            
#First we'll apply a fast GBM for quick first results and valuable insight in the data.


In [ ]:
######################################################################################################################
######################################################################################################################
##2.2 Imputation

train2.head(3)

#Checking for Na's in dataframe
train2.isnull().any()#Will return the feature with True or False,True means have missing value else False

## Split train/test
# create dataset
X = train2
# split into train test sets
X_train, X_test= train_test_split(X, train_size=0.70)
print(X_train.shape, X_test.shape)

# Prepare dataset
X_train['Type']='Train' #Create a flag for Train and Test Data set
X_test['Type']='Test'
fullData = pd.concat([X_train,X_test],axis=0) #Combined both Train and Test Data set

ID_col = ['PassengerId']
target_col = ["Survived"]
cat_cols = ['Name','Sex','Ticket','Cabin','Embarked']
num_cols= list(set(list(fullData.columns))-set(cat_cols)-set(ID_col)-set(target_col))
other_col=['Type'] #Test and Train Data set identifier

num_cat_cols = num_cols+cat_cols # Combined numerical and Categorical variables

#Create a new variable for each variable having missing value with VariableName_NA 
# and flag missing value with 1 and other with 0

for var in num_cat_cols:
     if fullData[var].isnull().any()==True:
        fullData[var+'_NA']=fullData[var].isnull()*1 

#Impute numerical missing values with mean
fullData[num_cols] = fullData[num_cols].fillna(fullData[num_cols].mean(),inplace=True)

#Impute categorical missing values with -9999
fullData[cat_cols] = fullData[cat_cols].fillna(value = -9999)


In [ ]:
## 2.3 Encoding numeric features 

for var in cat_cols:
 number = LabelEncoder()
 fullData[var] = number.fit_transform(fullData[var].astype('str'))

#Target variable is also a categorical so convert it
fullData["Survived"] = number.fit_transform(fullData["Survived"].astype('str'))

train=fullData[fullData['Type']=='Train']
test=fullData[fullData['Type']=='Test']

train['is_train'] = np.random.uniform(0, 1, len(train)) <= .75
Train, Validate = train[train['is_train']==True], train[train['is_train']==False]

features=list(set(list(fullData.columns))-set(ID_col)-set(target_col)-set(other_col))

x_train = Train[list(features)].values
y_train = Train["Survived"].values
x_validate = Validate[list(features)].values
y_validate = Validate["Survived"].values
x_test=test[list(features)].values


In [ ]:
## 2.4 Hyperparameter tuning 

# define the grid of values to search
grid = dict()
grid['n_estimators'] = [10, 50, 100, 500] 
grid['learning_rate'] = [0.0001, 0.001, 0.01, 0.1, 1.0]
grid['subsample'] = [0.5, 0.7, 1.0]
grid['max_depth'] = [3, 7, 9]
# define the evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=1, random_state=1)
# define the grid search procedure
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy')
# execute the grid search
grid_result = grid_search.fit(X_train, Y_train)
# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# summarize all scores that were evaluated
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


In [ ]:
######################################################################################################################
######################################################################################################################
#2.5) Optimized predictive model GBM 

# GBM classifier 
model = GradientBoostingClassifier(
        learning_rate=0.1, 
        n_estimators=100,
        max_depth=3, 
        min_samples_split=2, 
        min_samples_leaf=1, 
        subsample=1,
        max_features='sqrt', 
        random_state=10)

model.fit(X_train,Y_train)

pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

accuracy_train = accuracy_score(pred_train,Y_train)
accuracy_test = accuracy_score(pred_test,Y_test)

fpr, tpr, _ = metrics.roc_curve(np.array(Y_train), model.predict_proba(X_train)[:,1])
auc_train = metrics.auc(fpr,tpr)

fpr, tpr, _ = metrics.roc_curve(np.array(Y_test), model.predict_proba(X_test)[:,1])
auc_test = metrics.auc(fpr,tpr)

## Cross validation accuracy
cross_val_rfc = cross_val_score(estimator=GradientBoostingClassifier(), X=X_train, y=Y_train, cv=2, n_jobs=-1)

# R Cross validation accuracy
cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=1, random_state=1)
# evaluate the model on the dataset
n_scores = cross_val_score(model, X_train, Y_train, scoring='accuracy', cv=cv, n_jobs=-1)

#Print model report:
print("\nModel Report")
print('AUC:',(auc_test))
print("Cross Validation Accuracy : ",round(cross_val_rfc.mean() * 100 , 2),"%")
print(' R Cv Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
print("Cross Validation Accuracy: ")
print(classification_report(Y_test, pred_test))


In [ ]:
######################################################################################################################
##2.6) Prediction  

## Predictions

## ROC/AUC 

print("Roc : ",(fpr, tpr, _))
print("Auc : ",(auc_test))

## Confusion matrix
print("Confusion matrix : ", confusion_matrix(Y_test, pred_test))

## Crosstab
pd.crosstab(Y_train,pd.Series(pred_train),rownames=['ACTUAL'],colnames=['PRED'])

## cross validation accuracy
print("Cross Validation Accuracy : ",round(cross_val_rfc.mean() * 100 , 2),"%")

## R cross validation accuracy
print(' R Cv Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

# ROC AUC plot
#baseline model
baseline_roc_auc = roc_auc_score(Y_test, model.predict(X_test))
fprB, tprB, thresholdsB = roc_curve(Y_test, model.predict_proba(X_test)[:,1])
## #model 1
## model1_roc_auc = roc_auc_score(y_test, model1.predict(X_test))
## fpr1, tpr1, thresholds1 = roc_curve(y_test, model1.predict_proba(X_test)[:,1])
## #new tuned model 
## new_roc_auc = roc_auc_score(y_test, new.predict(X_test))
## fprnew, tprnew, thresholds_new = roc_curve(y_test, new.predict_proba(X_test)[:,1])

plt.figure()
plt.plot(fprB, tprB, label='GBM Baseline (area = %0.2f)' % baseline_roc_auc)
## plt.plot(fpr1, tpr1, label='GBM Model 1 (area = %0.2f)' % model1_roc_auc)
## plt.plot(fprnew, tprnew, label='GBM Final Model (area = %0.2f)' % new_roc_auc)

plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

In [ ]:
######################################################################################################################
######################################################################################################################
##3) Prediction 3 

#3.1) GBM with all effecting variables. 

In [ ]:
######################################################################################################################
######################################################################################################################
##3.2 Fast GBM 4

# Effecting variables >20
model = LGBMClassifier()
model.fit(X_train, Y_train)

fig, ax = plt.subplots(figsize=(12,18))
lgb.plot_importance(model, max_num_features=50, height=0.8, ax=ax)
ax.grid(False)
plt.title("LightGBM - Feature Importance", fontsize=15)
plt.show()

#Creating clean dataset
myvars = train[["TARGET","var3","saldo_medio_var5_hace3","saldo_medio_var5_hace2"]]
train1 = myvars

# Split train/test
X_train, X_test= train_test_split(train1, train_size=0.70)
print(X_train.shape, X_test.shape)

# Split features/dependent
Y_train = X_train["TARGET"]
X_train = X_train.drop("TARGET", axis=1)
Y_test = X_test["TARGET"]
X_test = X_test.drop("TARGET", axis=1)


# GBM classifier 1
model = GradientBoostingClassifier(
        learning_rate=0.1, 
        n_estimators=100,
        max_depth=3, 
        min_samples_split=2, 
        min_samples_leaf=1, 
        subsample=1,
        max_features='sqrt', 
        random_state=10)

model.fit(X_train,Y_train)

pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

accuracy_train = accuracy_score(pred_train,Y_train)
accuracy_test = accuracy_score(pred_test,Y_test)

fpr, tpr, _ = metrics.roc_curve(np.array(Y_train), model.predict_proba(X_train)[:,1])
auc_train = metrics.auc(fpr,tpr)

fpr, tpr, _ = metrics.roc_curve(np.array(Y_test), model.predict_proba(X_test)[:,1])
auc_test = metrics.auc(fpr,tpr)

## Cross validation accuracy
cross_val_rfc = cross_val_score(estimator=GradientBoostingClassifier(), X=X_train, y=Y_train, cv=2, n_jobs=-1)

# R Cross validation accuracy
cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=1, random_state=1)
# evaluate the model on the dataset
n_scores = cross_val_score(model, X_train, Y_train, scoring='accuracy', cv=cv, n_jobs=-1)

#Print model report:
print("\nModel Report")
print('AUC:',(auc_test))
print("Cross Validation Accuracy : ",round(cross_val_rfc.mean() * 100 , 2),"%")
print(' R Cv Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
print("Cross Validation Accuracy: ")
print(classification_report(Y_test, pred_test))


In [ ]:
######################################################################################################################
##3.3 Prediction

## Predictions

## ROC/AUC 

print("Roc : ",(fpr, tpr, _))
print("Auc : ",(auc_test))

## Confusion matrix
print("Confusion matrix : ", confusion_matrix(Y_test, pred_test))

## Crosstab
pd.crosstab(Y_train,pd.Series(pred_train),rownames=['ACTUAL'],colnames=['PRED'])

## cross validation accuracy
print("Cross Validation Accuracy : ",round(cross_val_rfc.mean() * 100 , 2),"%")

## R cross validation accuracy
print(' R Cv Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

# ROC AUC plot
#baseline model
baseline_roc_auc = roc_auc_score(Y_test, model.predict(X_test))
fprB, tprB, thresholdsB = roc_curve(Y_test, model.predict_proba(X_test)[:,1])
## #model 1
## model1_roc_auc = roc_auc_score(y_test, model1.predict(X_test))
## fpr1, tpr1, thresholds1 = roc_curve(y_test, model1.predict_proba(X_test)[:,1])
## #new tuned model 
## new_roc_auc = roc_auc_score(y_test, new.predict(X_test))
## fprnew, tprnew, thresholds_new = roc_curve(y_test, new.predict_proba(X_test)[:,1])

plt.figure()
plt.plot(fprB, tprB, label='GBM Baseline (area = %0.2f)' % baseline_roc_auc)
## plt.plot(fpr1, tpr1, label='GBM Model 1 (area = %0.2f)' % model1_roc_auc)
## plt.plot(fprnew, tprnew, label='GBM Final Model (area = %0.2f)' % new_roc_auc)

plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

In [ ]:
######################################################################################################################
######################################################################################################################
##4) Prediction 4 

#4.1) Ensembling 

#For the fourth prediction we'll use an ensemble model.

In [ ]:
######################################################################################################################
######################################################################################################################
##4.2) Stacking ensemble

In [ ]:
## Voting ensemble
model1 = LogisticRegression(random_state=1)
model2 = DecisionTreeClassifier(random_state=1)
model3 = KNeighborsClassifier()
model = VotingClassifier(estimators=[('lr', model1), ('dt', model2), ('knn', model3)], voting='hard')
model.fit(X_train,Y_train)
model.score(X_test,Y_test) 

In [ ]:
## Averaging ensemble
model1 = DecisionTreeClassifier()
model2 = KNeighborsClassifier()
model3= LogisticRegression()

model1.fit(X_train,Y_train)
model2.fit(X_train,Y_train)
model3.fit(X_train,Y_train)

pred1=model1.predict_proba(X_test)
pred2=model2.predict_proba(X_test)
pred3=model3.predict_proba(X_test)

finalpred=(pred1+pred2+pred3)/3


In [ ]:
## weighted Averaging ensemble
model1 = DecisionTreeClassifier()
model2 = KNeighborsClassifier()
model3= LogisticRegression()

model1.fit(X_train,Y_train)
model2.fit(X_train,Y_train)
model3.fit(X_train,Y_train)

pred1=model1.predict_proba(X_test)
pred2=model2.predict_proba(X_test)
pred3=model3.predict_proba(X_test)

finalpred=(pred1*0.3+pred2*0.3+pred3*0.4)

In [ ]:
## Stacking model
def Stacking(model,train,y,test,n_fold):
   folds=StratifiedKFold(n_splits=n_fold,random_state=1)
   test_pred=np.empty((test.shape[0],1),float)
   train_pred=np.empty((0,1),float)
   for train_indices,val_indices in folds.split(train,y.values):
      x_train,x_val=train.iloc[train_indices],train.iloc[val_indices]
      y_train,y_val=y.iloc[train_indices],y.iloc[val_indices]

      model.fit(X=x_train,y=y_train)
      train_pred=np.append(train_pred,model.predict(x_val))
      test_pred=np.append(test_pred,model.predict(test))
      return test_pred.reshape(-1,1),train_pred

In [ ]:
model1 = DecisionTreeClassifier(random_state=1)

test_pred1 ,train_pred1=Stacking(model=model1,n_fold=10, train=X_train,test=X_test,y=Y_train)

train_pred1=pd.DataFrame(train_pred1)
test_pred1=pd.DataFrame(test_pred1)

In [ ]:
model2 = KNeighborsClassifier()

test_pred2 ,train_pred2=Stacking(model=model2,n_fold=10,train=X_train,test=X_test,y=Y_train)

train_pred2=pd.DataFrame(train_pred2)
test_pred2=pd.DataFrame(test_pred2)

In [ ]:
df = pd.concat([train_pred1, train_pred2], axis=1)
df_test = pd.concat([test_pred1, test_pred2], axis=1)

model = LogisticRegression(random_state=1)
model.fit(df,Y_train)
model.score(df_test, Y_test)

In [ ]:
######################################################################################################################
######################################################################################################################
##5) Prediction 5

#5.1) EDA In-depth analysis

#For the fifth prediction we'll first do an in-depth EDA and then make predictions

In [ ]:
######################################################################################################################
######################################################################################################################
##6) Prediction 6 - Lean XGB prediction

#6.1) Lean XGB prediction

# For the sixth prediction we'll do a lean prediction with the often best performing
# algorithm XG Boost.

In [ ]:
# XGB classifier 
model = XGBClassifier(
        learning_rate=0.1, 
        n_estimators=100,
        max_depth=3, 
        min_samples_split=2, 
        min_samples_leaf=1, 
        subsample=1,
        max_features='sqrt', 
        random_state=10)

model.fit(X_train,Y_train)

pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

accuracy_train = accuracy_score(pred_train,Y_train)
accuracy_test = accuracy_score(pred_test,Y_test)

# from sklearn import metrics
# fpr, tpr, _ = metrics.roc_curve(np.array(Y_train), model.predict_proba(X_train)[:,1])
# auc_train = metrics.auc(fpr,tpr)

# fpr, tpr, _ = metrics.roc_curve(np.array(Y_test), model.predict_proba(X_test)[:,1])
# auc_test = metrics.auc(fpr,tpr)

 ## Cross validation accuracy
cross_val_rfc = cross_val_score(estimator=XGBClassifier(), X=X_train, y=Y_train, cv=2, n_jobs=-1)

# R Cross validation accuracy
cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=1, random_state=1)
# evaluate the model on the dataset
n_scores = cross_val_score(model, X_train, Y_train, scoring='accuracy', cv=cv, n_jobs=-1)


In [ ]:
######################################################################################################################
##5.7 Prediction

#Print model report:
# print("\nModel Report")
# print('AUC:',(auc_test))
print("Cross Validation Accuracy : ",round(cross_val_rfc.mean() * 100 , 2),"%")
print(' R Cv Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
# print("Cross Validation Accuracy: ")
# print(classification_report(Y_test, pred_test))

In [ ]:
######################################################################################################################
######################################################################################################################

# Optimization 1: 

# Optimization 2: 

# Optimization 3: 

# etc.